# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,leningradskiy,69.3833,178.4167,-12.89,93,100,2.31,RU,1680313521
1,1,coquimbo,-29.9533,-71.3436,15.85,94,0,3.60,CL,1680313522
2,2,landau,49.2075,8.1133,10.73,73,100,4.02,DE,1680313523
3,3,alta floresta,-9.8756,-56.0861,22.00,95,85,0.20,BR,1680313525
4,4,bluff,-46.6000,168.3333,16.27,71,10,1.03,NZ,1680313526


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
vacationMap = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
vacationMap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [60]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filtered_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)].dropna()

# Display sample data
city_data_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
57,57,port macquarie,-31.4333,152.9167,24.94,49,0,2.93,AU,1680313597
100,100,trincomalee,8.5711,81.2335,26.28,82,0,3.83,LK,1680313656
181,181,natal,-5.7950,-35.2094,25.36,88,0,2.06,BR,1680313769
302,302,taree,-31.9000,152.4667,24.56,45,0,2.65,AU,1680313938
331,331,praya,14.9215,-23.5087,23.30,68,0,1.03,CV,1680313977
404,404,maceio,-9.6658,-35.7353,23.69,94,0,1.54,BR,1680313842
531,531,sharjah,25.3573,55.4033,21.99,83,0,2.57,AE,1680314235
538,538,buin,-33.7333,-70.7500,24.13,29,0,0.75,CL,1680314251


### Step 3: Create a new DataFrame called `hotel_df`.

In [61]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
57,port macquarie,AU,-31.4333,152.9167,49,
100,trincomalee,LK,8.5711,81.2335,82,
181,natal,BR,-5.7950,-35.2094,88,
302,taree,AU,-31.9000,152.4667,45,
331,praya,CV,14.9215,-23.5087,68,
404,maceio,BR,-9.6658,-35.7353,94,
531,sharjah,AE,25.3573,55.4033,83,
538,buin,CL,-33.7333,-70.7500,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
params = {
    "categories":categories,
    "filter": "",
    "bias": "",
    "limit": 1,
    "apiKey":geoapify_key
}

url = ""

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lati = hotel_df.loc[index, 'Lat']
    lng = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lati},{radius}"
    params["bias"] = f"proximity:{lng},{lati}"
    
    # # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


# Display sample data
hotel_df

Starting hotel search
port macquarie - nearest hotel: The Observatory
trincomalee - nearest hotel: Silver Star Inn
natal - nearest hotel: Natal Palace Hotel
taree - nearest hotel: Wingham Hotel
praya - nearest hotel: Casa Sodadi
maceio - nearest hotel: Maceió Hostel e Pousada
sharjah - nearest hotel: shakeel bhai room
buin - nearest hotel: Hotel Casa Real


,City,Country,Lat,Lng,Humidity,Hotel Name
57,port macquarie,AU,-31.4333,152.9167,49,The Observatory
100,trincomalee,LK,8.5711,81.2335,82,Silver Star Inn
181,natal,BR,-5.7950,-35.2094,88,Natal Palace Hotel
302,taree,AU,-31.9000,152.4667,45,Wingham Hotel
331,praya,CV,14.9215,-23.5087,68,Casa Sodadi
404,maceio,BR,-9.6658,-35.7353,94,Maceió Hostel e Pousada
531,sharjah,AE,25.3573,55.4033,83,shakeel bhai room
538,buin,CL,-33.7333,-70.7500,29,Hotel Casa Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
%%capture --no-display

# Configure the map plot
vacationMapfiltered = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=['Hotel Name','Country'],
    alpha = 0.5
)

# Display the map
vacationMapfiltered

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)